In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx


In [2]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
import torch

2025-05-27 12:10:31.531661: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748347831.753371      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748347831.817157      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
writer = SummaryWriter()

In [4]:
!pip list

Package                            Version             Editable project location
---------------------------------- ------------------- -------------------------
absl-py                            1.4.0
accelerate                         1.5.2
aiofiles                           22.1.0
aiohappyeyeballs                   2.6.1
aiohttp                            3.11.18
aiosignal                          1.3.2
aiosqlite                          0.21.0
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.5
ale-py                             0.10.2
alembic                            1.15.2
altair                             5.5.0
annotated-types                    0.7.0
annoy                              1.17.3
ansicolors                         1.1.8
antlr4-python3-runtime             4.9.3
anyio                              4.9.0
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
args    

In [5]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="uer/roberta-base-finetuned-dianping-chinese"
)

out = classifier(["东西不错，份量手感，价格都没的说，点赞。关节阻尼合适。细节到位。","收到快递后都没拆封，外包装给人一种很廉价的感觉！"])
print(out)

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'positive (stars 4 and 5)', 'score': 0.9096200466156006}, {'label': 'negative (stars 1, 2 and 3)', 'score': 0.996224045753479}]


In [6]:
from transformers import AutoModel

bert = AutoModel.from_pretrained('google-bert/bert-base-chinese')
bert

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [7]:
import torch

token_indies = torch.randint(1,100,(4,10))

result = bert(token_indies)

last_hidden = result['last_hidden_state']
pooler_out = result['pooler_output']

print(last_hidden)
print(pooler_out)

tensor([[[-0.3460, -0.3563, -0.0734,  ...,  0.8236,  0.4229, -0.3124],
         [-0.2969, -0.4052, -0.0085,  ...,  0.7124,  0.4802, -0.4005],
         [-0.2099, -0.5120,  0.0189,  ...,  0.9002,  0.4324, -0.3310],
         ...,
         [-0.1661, -0.2909,  0.0994,  ...,  1.0647,  0.5215, -0.2407],
         [-0.1768, -0.3257,  0.0696,  ...,  1.0759,  0.5631, -0.2932],
         [-0.1859, -0.3856,  0.0392,  ...,  1.0393,  0.5387, -0.3069]],

        [[-0.2082, -0.2135,  0.5017,  ...,  0.9885,  0.5091, -0.2440],
         [-0.2212, -0.3326,  0.4889,  ...,  0.8581,  0.5836, -0.2668],
         [-0.1435, -0.4225,  0.5081,  ...,  1.0662,  0.5534, -0.2404],
         ...,
         [-0.0474, -0.1599,  0.5889,  ...,  1.1385,  0.6136, -0.1520],
         [-0.0635, -0.2034,  0.6072,  ...,  1.1679,  0.6516, -0.1689],
         [-0.0829, -0.2590,  0.6072,  ...,  1.1085,  0.6531, -0.2009]],

        [[-0.2878, -0.1325,  0.5426,  ...,  1.0396,  0.6838, -0.1203],
         [-0.3311, -0.1274,  0.5297,  ...,  0

In [8]:
from transformers import AutoConfig, BertModel

config = AutoConfig.from_pretrained('google-bert/bert-base-chinese',foo=1,bar=True)

print(config)

new_model = BertModel(config)
print(new_model)

print(bert.config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.51.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [10]:
s = "近年来，《新闻联播》不断进行内容和技术上的创新，如增加互动环节和多媒体报道形式，以适应新媒体时代的需求"

tokens = tokenizer.tokenize(s)
print(tokens)

# 文本转换词典中索引值
token_idx = tokenizer.encode(s)
print(token_idx)

# 解码
raw_txt = tokenizer.decode(token_idx)
print(raw_txt)

['近', '年', '来', '，', '《', '新', '闻', '联', '播', '》', '不', '断', '进', '行', '内', '容', '和', '技', '术', '上', '的', '创', '新', '，', '如', '增', '加', '互', '动', '环', '节', '和', '多', '媒', '体', '报', '道', '形', '式', '，', '以', '适', '应', '新', '媒', '体', '时', '代', '的', '需', '求']
[101, 6818, 2399, 3341, 8024, 517, 3173, 7319, 5468, 3064, 518, 679, 3171, 6822, 6121, 1079, 2159, 1469, 2825, 3318, 677, 4638, 1158, 3173, 8024, 1963, 1872, 1217, 757, 1220, 4384, 5688, 1469, 1914, 2054, 860, 2845, 6887, 2501, 2466, 8024, 809, 6844, 2418, 3173, 2054, 860, 3198, 807, 4638, 7444, 3724, 102]
[CLS] 近 年 来 ， 《 新 闻 联 播 》 不 断 进 行 内 容 和 技 术 上 的 创 新 ， 如 增 加 互 动 环 节 和 多 媒 体 报 道 形 式 ， 以 适 应 新 媒 体 时 代 的 需 求 [SEP]


In [11]:
corpus = ["Transformers 支持在 PyTorch、TensorFlow 和 JAX 上的互操作性",
          "这给在模型的每个阶段使用不同的框架带来了灵活性",
          "在一个框架中使用几行代码训练一个模型，然后在另一个框架中加载它并进行推理",]

model_inputs = tokenizer(corpus, # 训练语料集合
                        return_tensors='pt', # 生成数据类型  pt= pytorch
                        padding = True, # 语料中最长token序列构建矩阵.其余不足用padding填充
                        truncation = True, # 超过模型指定最大长度token序列, 裁剪
                       )
model_inputs



{'input_ids': tensor([[ 101,  100, 3118, 2898, 1762,  100,  510,  100, 1469,  100,  677, 4638,
          757, 3082,  868, 2595,  102,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0],
        [ 101, 6821, 5314, 1762, 3563, 1798, 4638, 3680,  702, 7348, 3667,  886,
         4500,  679, 1398, 4638, 3427, 3373, 2372, 3341,  749, 4130, 3833, 2595,
          102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0],
        [ 101, 1762,  671,  702, 3427, 3373,  704,  886, 4500, 1126, 6121,  807,
         4772, 6378, 5298,  671,  702, 3563, 1798, 8024, 4197, 1400, 1762, 1369,
          671,  702, 3427, 3373,  704, 1217, 6770, 2124, 2400, 6822, 6121, 2972,
         4415,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
bert(**model_inputs)

bert(input_ids = model_inputs['input_ids'],attention_mask = model_inputs['attention_mask'],)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0611,  0.4106, -1.2455,  ...,  1.3314,  0.0429, -0.0155],
         [ 0.2464,  0.4281, -0.6372,  ..., -0.0128, -0.3045, -0.4564],
         [ 0.9416, -0.5959, -1.1102,  ...,  0.8367,  0.6585, -0.1977],
         ...,
         [ 0.1383,  0.3651,  0.0559,  ...,  1.0691, -0.1486, -0.0040],
         [ 0.5306,  0.3890, -0.5643,  ...,  0.4631, -0.0893,  0.2267],
         [-0.0863,  0.4895, -0.0087,  ...,  1.0766, -0.0377, -0.1198]],

        [[ 0.3019,  0.1714, -0.3866,  ..., -0.1647, -0.7724, -0.0355],
         [ 0.2593,  0.2725,  0.6599,  ..., -1.4209, -0.4603, -0.0489],
         [ 0.9462, -0.7661,  0.3176,  ..., -0.8720,  0.1593,  0.0678],
         ...,
         [ 0.1626, -0.5553, -0.6020,  ..., -0.1075, -0.3675, -0.7436],
         [ 0.2843, -0.5141, -0.5088,  ..., -0.1068, -0.3596, -0.5010],
         [ 0.2921, -0.3505, -0.0686,  ..., -0.1541, -0.3286, -0.4379]],

        [[-0.3223, -0.0823,  0.0271,  ...,  0.2171, -

In [13]:
# 定制模型输出
from transformers import AutoModelForSequenceClassification,AutoModelForMaskedLM

# 完成文本分类任务(5个类别)
model1 = AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-chinese',num_labels=5)
print(model1)

# 完成bert预训练任务 MLM
model2 = AutoModelForMaskedLM.from_pretrained('google-bert/bert-base-chinese')
print(model2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mode

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [14]:
def build_collate(tokenizer):
    def collate_fn(batch):
        sentens,labels = zip(*batch)

        # tokenizer转换
        model_inputs = tokenizer(sentens,return_tensors='pt',padding =True,truncation = True)
        labels = torch.tensor(labels)

        return model_inputs,labels
    return collate_fn
    

In [15]:
import pandas as pd
df = pd.read_excel('/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx')
print(df.head())


          爬取时间(__time)                                        爬取链接(__url)  \
0  2019-03-08 00:50:34  https://sclub.jd.com/comment/productPageCommen...   
1  2019-03-08 00:50:34  https://sclub.jd.com/comment/productPageCommen...   
2  2019-03-08 00:50:34  https://sclub.jd.com/comment/productPageCommen...   
3  2019-03-08 00:50:34  https://sclub.jd.com/comment/productPageCommen...   
4  2019-03-08 00:50:33  https://sclub.jd.com/comment/productPageCommen...   

   商品ID(product_id)  评价时间(publish_time)  评分（总分5分）(score) 评价内容(content)  \
0           4722324          1550631798                5    此用户未填写评价内容   
1           4722324          1550633151                5    此用户未填写评价内容   
2           4722324          1550633330                3    此用户未填写评价内容   
3           4722324          1550633401                5    此用户未填写评价内容   
4           4722324          1550633461                5    此用户未填写评价内容   

  评价者(author_name) 评价者会员等级(author_level) 商品sku(product_sku) 评价标签(tags)  
0            j***e 

In [16]:
sc_data = df[['评价内容(content)','评分（总分5分）(score)']]

filter_data = sc_data[sc_data['评价内容(content)'] != '此用户未填写评价内容']
print(filter_data)

                                           评价内容(content)  评分（总分5分）(score)
15                                           一般般，一分钱一分货吧                1
18                        商品质量很好，很满意，配送速度快啊，而且配送员态度也非常好。                4
19                                                   。。。                5
22                            刘慧敏提莫摸摸摸休息泽TCL退咯的一组婆婆破鼓规土局                5
25                                      还好还好还好还好红红火火好很好好                5
...                                                  ...              ...
71813                                   还可以，挺合身！物流挺好的！！快                5
71814                                                还可以                5
71815                                    没有色差，穿上很舒服，到货快。                5
71816                                                 可以                5
71817  物有所值 客服贴心东西收到以后马上查看，发现与图片描述一致，超级喜欢，卖家发货 速度很快 ，...                5

[44422 rows x 2 columns]


In [17]:
filter_data.head()


,评价内容(content),评分（总分5分）(score)
15,一般般，一分钱一分货吧,1
18,商品质量很好，很满意，配送速度快啊，而且配送员态度也非常好。,4
19,。。。,5
22,刘慧敏提莫摸摸摸休息泽TCL退咯的一组婆婆破鼓规土局,5
25,还好还好还好还好红红火火好很好好,5


In [18]:
data = filter_data.values
data

array([['一般般，一分钱一分货吧', 1],
       ['商品质量很好，很满意，配送速度快啊，而且配送员态度也非常好。', 4],
       ['。。。', 5],
       ...,
       ['没有色差，穿上很舒服，到货快。', 5],
       ['可以', 5],
       ['物有所值 客服贴心东西收到以后马上查看，发现与图片描述一致，超级喜欢，卖家发货 速度很快 ，，服务也很到位，给老板点个赞，下次还会来购买.........',
        5]], dtype=object)

In [19]:
train ,test = train_test_split(data)
print(train.shape)
print(test.shape)

(33316, 2)
(11106, 2)


In [20]:
tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-bert-wwm')

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [21]:
def wpap_data(batch_data):
    comments,label_vals = zip(*batch_data) 
    labels = torch.tensor([int(val) - 1 for val in label_vals])
    comments = list(comments)

    input_idices = tokenizer(comments,
                            return_tensors='pt',
                            padding = True,
                            truncation = True,)
    return input_idices,labels


In [22]:

train_dl = DataLoader(train,batch_size=32,shuffle=True,collate_fn=wpap_data)
test_dl = DataLoader(test,batch_size=32,shuffle=False,collate_fn=wpap_data)

In [23]:
# for item in train_dl:
#     print(item)
#     break

In [24]:
model_exercise_1 = AutoModelForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm',num_labels=5)
model_exercise_2 = AutoModelForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm',num_labels=5)

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
model_exercise_1 = model_exercise_1.to(device)
model_exercise_2 = model_exercise_2.to(device)
model_exercise_2.bert.trainable = False

In [26]:
loss_fn1 = nn.CrossEntropyLoss()
optim1 = Adam(model_exercise_1.parameters(),lr=1e-4)

loss_fn2 = nn.CrossEntropyLoss()
optim2 = Adam(model_exercise_2.parameters(),lr=1e-3)

In [27]:
total_step = 0
model_exercise_1.train()
for epoch in range(3):
    pbar = tqdm(train_dl)
    for input_data,labels_data in pbar:
        model_exercise_1.zero_grad()
        datas = {k:v.to(device) for k,v in input_data.items()}
        labels = labels_data.to(device)

        result = model_exercise_1(**datas)
        loss = loss_fn1(result.logits,labels)

        pbar.set_description(f'epoch:{epoch} train_loss:{loss.item():.4f}')

        writer.add_scalar("Fine Tuning Train Loss", loss, total_step)
        total_step+=1

        loss.backward()
        optim1.step()

torch.save(model_exercise_1.state_dict(),'model_1.pt')

epoch:0 train_loss:0.3322:  91%|█████████ | 944/1042 [12:01<01:14,  1.31it/s]


RuntimeError: The size of tensor a (626) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
total_step = 0
model_exercise_2.train()
for epoch in range(3):
    pbar = tqdm(train_dl)
    for input_data,labels_data in pbar:
        model_exercise_2.zero_grad()
        datas = {k:v.to(device) for k,v in input_data.items()}
        labels = labels_data.to(device)

        result = model_exercise_2(**datas)
        loss = loss_fn2(result.logits,labels)

        pbar.set_description(f'epoch:{epoch} train_loss:{loss.item():.4f}')

        writer.add_scalar("Fine Tuning Train Loss", loss, total_step)
        total_step+=1

        loss.backward()
        optim2.step()

torch.save(model_exercise_2.state_dict(),'model_2.pt')

In [ ]:
model_exercise_1.eval()
model_exercise_2.eval()

pbar_test = tqdm(test_dl)

for input_data,labels_data in pbar_test:
    datas = { k:v.to(device) for k,v in input_data.items() }
    labels = labels_data.to(device)

    with torch.no_grad():
        result1 = model_exercise_1(**datas)
        result2 = model_exercise_2(**datas)

    predict1 = torch.argmax(result1.logits, dim=-1)
    predict2 = torch.argmax(result2.logits, dim=-1)

    correct1 += (predict1 == labels).sum()
    correct2 += (predict1 == labels).sum()

In [ ]:
model_1.load_state_dict()